<a href="https://colab.research.google.com/github/VijayRameshkumar/AI_Curriculum/blob/master/deepAR_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
! pip install --upgrade mxnet-cu101
! pip install gluonts==0.7.5

# !pip install --upgrade mxnet-cu110

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd

# url = '/content/drive/MyDrive/PM_data/RX_util_huawei.csv'
# url = '/content/drive/MyDrive/PM_data/Temp_3KPI_1.csv'
# url = '/content/drive/MyDrive/RX_hu_Sample.csv'
url = '/content/drive/MyDrive/RX_NEC_Sample.csv'

df = pd.read_csv(url, date_parser=True)
df

FileNotFoundError: ignored

In [ ]:
df.object_id.value_counts()

In [ ]:
import pandas as pd
import numpy as np
import mxnet as mx
import matplotlib.pyplot as plt
import matplotlib as mpl
import argparse

from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.dataset.common import ListDataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.util import to_pandas
from gluonts.mx.trainer.model_averaging import ModelAveraging, SelectNBestSoftmax, SelectNBestMean
from gluonts.mx.trainer.learning_rate_scheduler import LearningRateReduction
from tqdm import tqdm
import math

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False
print("Number GPU's: " + str(mx.context.num_gpus()))

class DeepAR():

    def __init__(self, isObj, data, context_length, prediction_length, freq, KPI_ID, epoch, datetime, kpi_value='kpi_value'):
        self.data = data
        self.isObj = isObj
        self.context_length = context_length
        self.prediction_length = prediction_length
        self.freq = freq
        self.NO_NES = 1
        self.KPI_ID = KPI_ID
        self.epoch = epoch
        self.kpi_value = kpi_value
        self.cols = None
        self.tss = None
        self.forecast = None
        self.objects = None
        self.nodes = None
        self.datetime = datetime

        if self.datetime == 'nrt':
            self.datetime = 'ne_datetime'
            self.cols = "sub_ne_type,ne_id,object_id,kpi_value,max_value,min_value,kpi_id,algorithm,type,ne_datetime,ne_date,fw_kpi_value,fw_max_value,fw_min_value,confidence".split(",")

        elif self.datetime == 'Days':
            self.datetime = 'ne_date'
            self.cols = 'sub_ne_type,ne_id,object_id,kpi_value,max_value,min_value,kpi_id,algorithm,type,ne_date,fw_kpi_value,fw_max_value,fw_min_value,confidence'.split(',')

        elif self.datetime=='Hours':
            self.cols = 'sub_ne_type,ne_id,object_id,kpi_value,max_value,min_value,fw_kpi_value,fw_max_value,fw_min_value,kpi_id,algorithm,type,ne_datetime,ne_date,confidence'.split(',')
            self.datetime = 'ne_datetime'

    def impute(self, df):
        print(f"Before total number of nan in dataset {df.isnull().sum().sum()}")
        if df.isnull().sum().sum() != 0:
            for i in df.columns:
                df[i] = df[i].interpolate(limit=260, method='spline', order=1, limit_direction='forward')
                df[i] = df[i].fillna(df[i].mean())
        print(f"After total number of nan in dataset {df.isnull().sum().sum()}")
        return df

    def preprocessing(self, ):
        print("I'm in preprocessing : ")
        df = self.data

        if df.isnull().sum().sum() != 0:
            df = self.impute(df)

        if self.isObj:
            self.objects = df.object_id.values.tolist()
            self.nodes = df.ne_id.values.tolist()
            df = pd.pivot_table(index='object_id', columns=self.datetime, values=self.kpi_value, data=df)
        else:
            self.nodes = df.ne_id.values.tolist()
            df = pd.pivot_table(index='ne_id', columns=self.datetime, values=self.kpi_value, data=df)
        print(f"df shape {df.shape}")
        self.data = df

    def get_feature_label(self, ):
        return self.data.index.astype('category').codes.astype('int')

    def train_test_split(self, ):
        self.preprocessing()
        print("I'm in train test split : ")
        df_train = self.data
        df_test = self.data.iloc[:, -self.context_length:]

        self.NO_NES = df_train.shape[0]
        start_train = pd.Timestamp(df_train.columns[0], freq=self.freq)
        start_test = pd.Timestamp(df_test.columns[0], freq=self.freq)

        print(f"train dataset shape : {df_train.shape}\ntest dataset shape : {df_test.shape}")
        return df_train.values, df_test.values, start_train, start_test

    def prepare_dataset(self, ):
        df_train, df_test, start_train, start_test = self.train_test_split()
        # print("I'm in before ts_code")
        ts_code = self.get_feature_label()

        # print("I'm in before train_ds")

        train_ds = ListDataset([
                        {
                            FieldName.TARGET : target,
                            FieldName.START : start_train,
                            FieldName.FEAT_STATIC_CAT:fsc
                        }
                        for (target, fsc) in zip(df_train[:], ts_code[:].reshape(-1, 1))
                        ], freq=self.freq)

        # print("I'm in before test_ds")

        test_ds = ListDataset([
                                {
                                    FieldName.TARGET : target,
                                    FieldName.START : start_test,
                                    FieldName.FEAT_STATIC_CAT:fsc
                                }
                                for (target, fsc) in zip(df_test, ts_code.reshape(-1, 1))
                                ], freq=self.freq)
        return train_ds, test_ds

    def model_instantiate(self, ):
        callbacks = [
                     LearningRateReduction(objective="min",patience=2,
                                           base_lr=1e-4,
                                           decay_factor=0.6),
                     ModelAveraging(avg_strategy=SelectNBestMean(num_models=2))]

        estimator = DeepAREstimator(mx.context.gpu(0),
                            freq=self.freq,
                            context_length=self.context_length,
                            prediction_length=self.prediction_length,
                            num_layers=8,
                            num_cells=256,
                            use_feat_static_cat=True,
                            cardinality=[1],
                            cell_type='gru',
                            trainer=Trainer(epochs=self.epoch, callbacks=callbacks),
                            batch_size=128,
                            impute_missing_values=True,
                            num_imputation_samples=3)
        return estimator

    def sample_df(self, forecast):
        samples = forecast.samples
        ns, h = samples.shape
        dates = pd.date_range(forecast.start_date, freq=forecast.freq, periods=h)
        return pd.DataFrame(samples.T, index=dates)

    def model_train(self, ):
        print("dataset preparation :")
        train_ds, test_ds = self.prepare_dataset()

        print("model instantiate :")
        estimator = self.model_instantiate()

        print("model training :")
        predictor = estimator.train(training_data=train_ds)

        print("forecasting")
        forecast_it, ts_it = make_evaluation_predictions(dataset = test_ds, predictor=predictor, num_samples=1)
        forecast_it_, ts_it_ = make_evaluation_predictions(dataset = train_ds, predictor=predictor, num_samples=1)

        print("obtaining time series conditionning values ...")
        tss = list(tqdm(ts_it, total=len(test_ds)))
        tss_ = list(tqdm(ts_it_, total=len(train_ds)))

        print("obtaining time series prediction")
        forecasts = list(tqdm(forecast_it, total=len(test_ds)))
        forecasts_ = list(tqdm(forecast_it_, total=len(train_ds)))
        # forecasts = self.sample_df(forecasts)

        print("Evaluation")
        evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
        agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds))
        agg_metrics_, item_metrics_ = evaluator(iter(tss_), iter(forecasts_), num_series=len(train_ds))

        print(f"TRAIN MEAN MAE : {item_metrics_.abs_error.mean()}")
        print(f"TEST MEAN MAE : {item_metrics.abs_error.mean()}\n")

        print(f"TRAIN MEAN MSE : {item_metrics_.MSE.mean()}")
        print(f"TEST MEAN MSE : {item_metrics.MSE.mean()}\n")

        print(f"TRAIN MEAN RMSE : {math.sqrt(item_metrics_.MSE.mean())}")
        print(f"TEST MEAN RMSE : {math.sqrt(item_metrics.MSE.mean())}\n")

        print(f"TRAIN MEAN MAPE : {item_metrics_.MAPE.mean()}")
        print(f"TEST MEAN MAPE : {item_metrics.MAPE.mean()}")

        self.tss = tss
        self.forecast = forecasts

        # forecasts_ = list(forecast_it_)
        # tss_ = list(ts_it_)
        # ts_entry_ = tss_[0]
        # forecast_entry_ = forecasts_[0]

        # forecasts = list(forecast_it)
        # tss = list(ts_it)
        # ts_entry = tss[0]
        # forecast_entry = forecasts[0]

        return predictor, train_ds, test_ds, (tss_, forecasts_), (tss, forecasts)

    def get_sma(self, values, rate):
        return values.rolling(rate).mean()

    def get_bollinger_bands(self, values, rate=2):
        sma = self.get_sma(values, rate)
        std = values.rolling(rate).std()
        bollinger_up = sma + std * 2 # Calculate top band
        bollinger_down = sma - std * 2 # Calculate bottom band
        return bollinger_up, bollinger_down

    def predict(self, ):
        predictor, train_ds, test_ds, train, test = self.model_train()
        dt = []

        for index, i in tqdm(enumerate(list(predictor.predict(test_ds, num_samples=1)))):
            ts_entry = self.sample_df(i).reset_index().rename(columns={'index' : 'ne_datetime'})
            ts_entry.columns = ['ne_date_time', 'kpi_value']

            ts_entry['ne_date_time'] = pd.to_datetime(ts_entry['ne_date_time'])
            ts_entry['ne_date'] = ts_entry['ne_date_time'].dt.date
            ts_entry['kpi_value'] = ts_entry['kpi_value'].astype('float32').abs()
            ts_entry['kpi_value'].fillna(value=ts_entry['kpi_value'].mean(), inplace=True)
            max, min = self.get_bollinger_bands(ts_entry['kpi_value'])
            ts_entry['max_value'] = pd.Series(max)
            ts_entry['min_value'] = pd.Series(min)
            ts_entry['min_value'].fillna(value=ts_entry['min_value'].mean(), inplace=True)
            ts_entry['max_value'].fillna(value=ts_entry['max_value'].mean(), inplace=True)
            ts_entry['kpi_id'] = self.KPI_ID
            ts_entry['algorithm'] = 'NeuralNetwork'

            if self.isObj:
                ts_entry['object_id'] = self.objects[index]
                ts_entry['ne_id'] = self.nodes[index]
                ts_entry['type'] = 0
                ts_entry['fw_kpi_value'] = 0
                ts_entry['fw_max_value'] = 0
                ts_entry['fw_min_value'] = 0
                ts_entry['confidence'] = 0.95
            else:
                ts_entry['object_id'] = '-'
                ts_entry['ne_id'] = self.nodes[index]
                ts_entry['type'] = 1
                ts_entry['fw_kpi_value'] = 0
                ts_entry['fw_max_value'] = 0
                ts_entry['fw_min_value'] = 0
                ts_entry['confidence'] = 0.95

            ts_entry['sub_ne_type'] = ""
            dt.append(ts_entry)

        dt = pd.concat(dt).reset_index(drop=True)
        cols = self.cols
        return dt[cols], train_ds, test_ds, predictor, train, test

# if __name__ == '__main__':

#     url = '/content/drive/MyDrive/RX_hu_Sample.csv'
#     df = pd.read_csv(url, date_parser=True)
#     result = DeepAR(isObj=True, data=df, context_length=5, prediction_length=3, freq='D', KPI_ID='21691544429562887',
#                     epoch=2, datetime='Days', kpi_value='value')
#     T = r.predict()

/usr/local/lib/python3.7/dist-packages/gluonts/json.py:46: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "


Number GPU's: 1


In [ ]:
from gluonts.dataset.util import to_pandas
# from DeepAR import DeepAR

# from DeepAR import DeepAR
# '1D' - 1 Day
# '1H' - 1 Hour
# '1M' - 1 Month
# '1Y' - 1 Year
# '1s' - 1 Second
# '1min' - 1 Minute


r = DeepAR(isObj=True, data=df, context_length=10, prediction_length=5, freq='D', KPI_ID='RX_NEC',
           epoch=5, datetime='Days', kpi_value='kpi_value')

dt, train_ds, test_ds, predictor, train, test = r.predict()

dataset preparation :
I'm in preprocessing : 
df shape (3, 15)
I'm in train test split : 
train dataset shape : (3, 15)
test dataset shape : (3, 10)
model instantiate :
model training :


100%|██████████| 50/50 [00:09<00:00,  5.20it/s, epoch=5/5, avg_epoch_loss=-.0984]


forecasting
obtaining time series conditionning values ...


100%|██████████| 3/3 [00:00<00:00, 1953.56it/s]


obtaining time series prediction


100%|██████████| 3/3 [00:00<00:00,  5.32it/s]


Evaluation


Running evaluation: 100%|██████████| 3/3 [00:00<00:00, 52.04it/s]/usr/local/lib/python3.7/dist-packages/gluonts/evaluation/metrics.py:102: RuntimeWarning: divide by zero encountered in double_scalars
  return np.mean(np.abs(target - forecast)) / seasonal_error
/usr/local/lib/python3.7/dist-packages/gluonts/evaluation/metrics.py:150: RuntimeWarning: divide by zero encountered in double_scalars
  return numerator / seasonal_error

/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:1625: UserWarning: Warning: converting a masked element to nan.
  subarr = np.array(values, dtype=dtype, copy=copy)
Running evaluation: 100%|██████████| 3/3 [00:00<00:00, 68.00it/s]
/usr/local/lib/python3.7/dist-packages/gluonts/evaluation/metrics.py:102: RuntimeWarning: divide by zero encountered in double_scalars
  return np.mean(np.abs(target - forecast)) / seasonal_error
/usr/local/lib/python3.7/dist-packages/gluonts/evaluation/metrics.py:150: RuntimeWarning: divide by zero encountered in dou

TRAIN MEAN MAE : 2.2919697364171348
TEST MEAN MAE : 3.1934796969095864

TRAIN MEAN MSE : 0.5093118717273076
TEST MEAN MSE : 0.7678727596998214

TRAIN MEAN RMSE : 0.7136608940717627
TEST MEAN RMSE : 0.8762834927692187

TRAIN MEAN MAPE : 0.3641509175300598
TEST MEAN MAPE : 0.6333005666732788


3it [00:00, 74.04it/s]


In [ ]:
def sample_df(forecast):
    samples = forecast.samples
    ns, h = samples.shape
    dates = pd.date_range(forecast.start_date, freq=forecast.freq, periods=h)
    return pd.DataFrame(samples.T, index=dates)

In [ ]:
print(df[df.object_id == 'SUM-LA-SDN-0558_1-0-0-0-8519680'].kpi_value.mean(),
df[df.object_id == 'SUM-LA-SDN-0558_1-0-0-0-8585216'].kpi_value.mean(),
df[df.object_id == 'SUM-LA-SDN-0558_1-0-0-0-8650752'].kpi_value.mean())
# df['SUM-LA-SDN-0558_1-0-0-0-8650752']

0.9893333333333333 2.186 0.0


In [ ]:
sample_df(train[1][0])

,0
2021-10-07,1.602985
2021-10-08,1.066790
2021-10-09,0.498158
2021-10-10,1.504086
2021-10-11,1.116071


In [ ]:
t = train[0][0]
t['a'] = df[df.object_id == 'SUM-LA-SDN-0558_1-0-0-0-8519680'].kpi_value.reset_index(drop=True).values
t

,0,a
2021-09-27,1.08,1.08
2021-09-28,1.60,1.60
2021-09-29,1.51,1.51
2021-09-30,1.28,1.28
2021-10-01,1.02,1.02
2021-10-02,0.00,0.00
2021-10-03,0.00,0.00
2021-10-04,1.20,1.20
2021-10-05,1.09,1.09
2021-10-06,0.86,0.86


In [ ]:
t = train[0][1]
t['a'] = df[df.object_id == 'SUM-LA-SDN-0558_1-0-0-0-8585216'].kpi_value.reset_index(drop=True).values
t

,0,a
2021-09-27,1.99,1.99
2021-09-28,2.79,2.79
2021-09-29,2.42,2.42
2021-09-30,1.74,1.74
2021-10-01,2.21,2.21
2021-10-02,2.41,2.41
2021-10-03,2.17,2.17
2021-10-04,2.17,2.17
2021-10-05,2.38,2.38
2021-10-06,2.30,2.30


In [ ]:
t = train[0][2]
t['a'] = df[df.object_id == 'SUM-LA-SDN-0558_1-0-0-0-8650752'].kpi_value.reset_index(drop=True).values
t

,0,a
2021-09-27,0.0,0.0
2021-09-28,0.0,0.0
2021-09-29,0.0,0.0
2021-09-30,0.0,0.0
2021-10-01,0.0,0.0
2021-10-02,0.0,0.0
2021-10-03,0.0,0.0
2021-10-04,0.0,0.0
2021-10-05,0.0,0.0
2021-10-06,0.0,0.0


In [ ]:
def plot_prob_forecasts(ts_entry, forecast_entry, actual_entry):
    plot_length = 150
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='r')
    actual_entry.plot(color='g')
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    plt.show()

In [ ]:
for i in train[1]:
    print(sample_df(i))

In [ ]:
o = df.object_id.unique()

dfs = []
for ind, i in enumerate(list(predictor.predict(train_ds, num_samples=1))):
    t = sample_df(i)
    t['object_id'] = o[ind]
    t.columns = ['pred', 'object_id']
    dfs.append(t)

pd.concat(dfs)

In [ ]:
for i in df.object_id.unique():
    plot_length = 150
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    t = df[df.object_id == i][['ne_date', 'kpi_value']]
    t.set_index('ne_date')
    t.plot(color='g')
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    # print(df[df.object_id == i].kpi_value)
    # break
    # plot_prob_forecast(pd.to_datetime(df[df.object_id == i].ne_date).dt.day, df[df.object_id == i].kpi_value, str(i))

In [ ]:
next(iter(train_ds))

In [ ]:
list(test_ds)